In [ ]:
import jupyanno as ja
import numpy as np

In [ ]:
import traitlets as tr
import ipywidgets as widgets
class CornerstoneWidget(widgets.DOMWidget):
    _view_name = tr.Unicode('CornerstoneWidget').tag(sync=True)
    _view_module = tr.Unicode('cs_widget').tag(sync=True)
    _view_module_version = tr.Unicode('0.1.0').tag(sync=True)
    title_field = tr.Unicode('Awesome Widget').tag(sync=True)
    img_bytes = tr.Unicode('AQAAAAAAAAABAAAAAAAAAAEA').tag(sync=True)
    img_width = tr.Int(3).tag(sync=True)
    img_height= tr.Int(3).tag(sync=True)
    img_min = tr.Float(0).tag(sync=True)
    img_max = tr.Float(255).tag(sync=True)
    img_scale = tr.Float(1.0).tag(sync=True)
    
    def update_image(self, in_image):
        (self.img_width, self.img_height) = in_image.shape
        self.img_min = float(in_image.min())
        self.img_max = float(in_image.max())
        self.img_bytes = ja.cornerstone.encode_numpy_b64(in_image)
        self.img_scale=1.0

In [ ]:
%%javascript
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  },
    shim: {
        'cornerstone-core': {
            exports: 'cornerstone',
            deps: ['jquery']
        }
    }
});

In [ ]:
%%javascript
require.undef('cs_widget');
define('cs_widget', ["@jupyter-widgets/base", "cornerstone-core",
                    'cornerstoneMath', 'cornerstoneTools'], 
       function(widgets, cs, cm, ctools) {
            ctools.external.cornerstone = cs;
            ctools.external.cornerstoneMath = cm;
    var CornerstoneWidget = widgets.DOMWidgetView.extend({

        render: function() {
            this.message = document.createElement('div')
            this.viewer = document.createElement('div')
            var fv = $(this.viewer)
            fv.width('512px');
            fv.height('512px');
            // Enable our tools
            this.el.appendChild(this.message);
            this.el.appendChild(this.viewer);
            this.model.on('change:img_bytes', this.dicom_changed, this);
            this.model.on('change:img_scale', this.zoom_changed, this);
            this.model.on('change:title_field', this.message_changed, this);
        },
        
        parse_image: function(imageB64Data, width, height, min_val, max_val) {
            function str2ab(str) {
                var buf = new ArrayBuffer(str.length*2); // 2 bytes for each char
                var bufView = new Uint16Array(buf);
                var index = 0;
                for (var i=0, strLen=str.length; i<strLen; i+=2) {
                    var lower = str.charCodeAt(i);
                    var upper = str.charCodeAt(i+1);
                    bufView[index] = lower + (upper <<8);
                    index++;
                }
                return bufView;
            }
            function parsePixelData(base64PixelData, width, height)
            {
                var pixelDataAsString = window.atob(base64PixelData);
                var pixelData = str2ab(pixelDataAsString);
                return pixelData;
            }
            var imagePixelData = parsePixelData(imageB64Data);
            console.log('decoding: '+width+'x'+height+' => '+imagePixelData.length)
            function getPixelData() {
                return imagePixelData;
            }
            return {
                imageId: 'imageId',
                minPixelValue: min_val,
                maxPixelValue: max_val,
                slope: 1.0,
                intercept: 0,
                windowCenter : 0.5*(max_val+min_val),
                windowWidth : 0.5*(max_val-min_val),
                getPixelData: getPixelData,
                rows: width,
                columns: height,
                height: height,
                width: width,
                color: false,
                columnPixelSpacing: 1.0,
                rowPixelSpacing: 1.0,
                sizeInBytes: height * width * 2
            };
        },
        
        message_changed: function() {
            this.message.textContent = this.model.get('title_field');
        },
        
        dicom_changed: function() {
            var img_bytes = this.model.get('img_bytes')
            var img_width = this.model.get('img_width')
            var img_height = this.model.get('img_height')
            var img_min = this.model.get('img_min')
            var img_max = this.model.get('img_max')
            var out_img = this.parse_image(img_bytes, img_width, img_height, img_min, img_max);
            cs.enable(this.viewer);
            this.viewport = cs.getDefaultViewportForImage(this.viewer, out_img);
            console.log(out_img);
            cs.displayImage(this.viewer, out_img, this.viewport);
            ctools.mouseInput.enable(this.viewer);
            ctools.mouseWheelInput.enable(this.viewer);
            ctools.wwwc.activate(this.viewer, 1); // Left Click
            ctools.pan.activate(this.viewer, 2); // Middle Click
            ctools.zoom.activate(this.viewer, 4); // Right Click
            ctools.zoomWheel.activate(this.viewer); // Mouse Wheel
        },
        
        zoom_changed: function() {
            this.viewport.scale = this.model.get('img_scale');
            cs.setViewport(this.viewer, this.viewport);
        }
    });

    return {
        CornerstoneWidget : CornerstoneWidget
    };
});

In [ ]:
cs_view = CornerstoneWidget()
cs_view.title_field = 'Cornerstone Widget'

In [ ]:
from IPython.display import Javascript, display
import ipywidgets as ipw
size_scroller = ipw.IntSlider(value=128, min=3, max=2048, description='Image Size')
def show_image(cs_obj, img_maker):
    c_wid = size_scroller.value
    cs_obj.update_image(img_maker(c_wid))

def zoom_viewer(cs_obj, zf):
    cs_obj.img_scale+=zf

In [ ]:
noisy_img_but = ipw.Button(description='Noisy Image')
noisy_image = lambda x: np.random.uniform(-1000, 1000, size=(x, x))
noisy_img_but.on_click(lambda *args: show_image(cs_view, 
                                                noisy_image))

gradient_img_but = ipw.Button(description='Gradient Image')
gradient_image = lambda x: np.arange(x*x).reshape((x, x))
gradient_img_but.on_click(lambda *args: show_image(cs_view, 
                                                gradient_image))

half_img_but = ipw.Button(description='Half Image')
half_image = lambda x: np.eye(x)[:x//2]
half_img_but.on_click(lambda *args: show_image(cs_view, half_image))

zoom_in_but = ipw.Button(description='Zoom In')
zoom_in_but.on_click(lambda *args: zoom_viewer(cs_view, 0.25))
zoom_out_but = ipw.Button(description='Zoom Out')
zoom_out_but.on_click(lambda *args: zoom_viewer(cs_view, -0.25))

In [ ]:
ipw.VBox([
    cs_view,
    ipw.HBox([
        size_scroller,
        ipw.VBox([noisy_img_but, half_img_but, gradient_img_but]),
        ipw.VBox([zoom_in_but, zoom_out_but])
    ])
])